In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv('UpdatedResumeDataSet[1].csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['Category'].value_counts()

In [ ]:
sns.set_style("whitegrid")
total = len(df)
role_counts = df['Category'].value_counts()
plt.figure(figsize=(18,16))
plt.subplot(2, 1, 1)
ax = sns.countplot(
    y='Category',
    data=df,
    order=role_counts.index)
for p in ax.patches:
    percent = f'{(p.get_width() / total) * 100:.1f}%'
    ax.text(
        p.get_width() + 0.5,
        p.get_y() + p.get_height() / 2,
        percent,
        va='center')
plt.title("All Job Roles Distribution (Bar Chart)")
plt.xlabel("Count")
plt.ylabel("Job Role")
plt.subplot(2, 1, 2)
plt.pie(
    role_counts.values,
    labels=role_counts.index,
    autopct='%1.1f%%',
    startangle=140)
plt.title("All Job Roles Distribution (Pie Chart)")
plt.axis('equal')
plt.tight_layout()
plt.show()

In [ ]:
sns.set_style("whitegrid")
top10 = df['Category'].value_counts().head(10)
total_top10 = top10.sum()
plt.figure(figsize=(12,14))
plt.subplot(2, 1, 1)
ax = sns.barplot(
    x=top10.values,
    y=top10.index)
for i, v in enumerate(top10.values):
    percent = f'{(v / total_top10) * 100:.1f}%'
    ax.text(v + 0.5, i, percent, va='center')
plt.title("Top 10 Job Categories (Bar Graph)")
plt.xlabel("Count")
plt.ylabel("Category")
plt.subplot(2, 1, 2)
plt.pie(
    top10.values,
    labels=top10.index,
    autopct='%1.1f%%',
    startangle=140,
    wedgeprops={'edgecolor': 'black'})
plt.title("Top 10 Job Categories (Pie Chart)")
plt.axis('equal')
plt.tight_layout()
plt.show()


In [ ]:
df['Category'][0]


In [ ]:
df['Resume'][0]

In [ ]:
import re
def cleanResume(text):
    if not isinstance(text, str):
        return ""     
    text = re.sub(r'http\S+', ' ', text)          # remove URLs
    text = re.sub(r'RT|cc', ' ', text)             # remove RT, cc
    text = re.sub(r'#\S+', ' ', text)              # remove hashtags
    text = re.sub(r'@\S+', ' ', text)              # remove mentions
    text = re.sub(r'[^\x00-\x7f]', ' ', text)      # remove non-ASCII
    text = re.sub(r'[!"#$%&\'()*+,-./:;<=>?@[\\\]^_`{|}~]', ' ', text)
    text = re.sub(r'\s+', ' ', text)               # remove extra spaces
    return text.lower()


In [ ]:
df['Resume'] = df['Resume'].apply(cleanResume)

In [ ]:
df['Resume'][0]

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df['Category'])
df['Category'] = le.transform(df['Category'])

In [ ]:
df.Category.unique()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')

tfidf.fit(df['Resume'])
requredText  = tfidf.transform(df['Resume'])

In [ ]:
requredText

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(requredText, df['Category'], test_size=0.2, random_state=42)
X_train.shape

In [ ]:
X_test.shape

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Ensure that X_train and X_test are dense if they are sparse
X_train = X_train.toarray() if hasattr(X_train, 'toarray') else X_train
X_test = X_test.toarray() if hasattr(X_test, 'toarray') else X_test

# 1. Train KNeighborsClassifier
knn_model = OneVsRestClassifier(KNeighborsClassifier())
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)
print("\nKNeighborsClassifier Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_knn):.4f}")
print(f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred_knn)}")
print(f"Classification Report:\n{classification_report(y_test, y_pred_knn)}")

In [ ]:
svc_model = OneVsRestClassifier(SVC())
svc_model.fit(X_train, y_train)
y_pred_svc = svc_model.predict(X_test)
print("\nSVC Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_svc):.4f}")
print(f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred_svc)}")
print(f"Classification Report:\n{classification_report(y_test, y_pred_svc)}")

In [ ]:
rf_model = OneVsRestClassifier(RandomForestClassifier())
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print("\nRandomForestClassifier Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred_rf)}")
print(f"Classification Report:\n{classification_report(y_test, y_pred_rf)}")

In [ ]:
import pickle
pickle.dump(tfidf,open('tfidf.pkl','wb'))
pickle.dump(svc_model, open('clf.pkl', 'wb'))
pickle.dump(le, open("encoder.pkl",'wb'))

In [ ]:
# Function to predict the category of a resume
def pred(input_resume):
    # Preprocess the input text (e.g., cleaning, etc.)
    cleaned_text = cleanResume(input_resume) 

    # Vectorize the cleaned text using the same TF-IDF vectorizer used during training
    vectorized_text = tfidf.transform([cleaned_text])
    
    # Convert sparse matrix to dense
    vectorized_text = vectorized_text.toarray()

    # Prediction
    predicted_category = svc_model.predict(vectorized_text)

    # get name of predicted category
    predicted_category_name = le.inverse_transform(predicted_category)

    return predicted_category_name[0]  # Return the category name

In [ ]:
myresume = """
SYED IMTHIYAZ
+91 9550730829
Gurramkonda, Madanapalle, Andhra Pradesh
syedimthiyaz456@gmail.com
linkedin.com/in/syed-imthiyaz
github.com/imthiyaz-official

PROFESSIONAL SUMMARY
Computer Science undergraduate with strong skills in Python, SQL, and data analysis.
Experienced in exploratory data analysis (EDA), data cleaning, and feature engineering,
with hands-on experience using Pandas, NumPy, Matplotlib, and Seaborn.
Proficient in building interactive dashboards using Power BI and Tableau, and familiar with
AWS cloud services (EC2, S3, IAM).
Seeking entry-level Data Analyst or Data Science roles to apply analytical,
statistical, and problem-solving skills to real-world business problems.

SKILLS
Programming: Python, Java (Basics)
Data Analysis: EDA, Data Wrangling, Data Cleaning, Feature Engineering
Statistics: Descriptive Statistics, Probability, Hypothesis Testing
Machine Learning: Regression, Classification, Model Evaluation
Databases: SQL (Joins, Subqueries, Aggregations), PostgreSQL
Visualization: Power BI, Tableau, Seaborn
Libraries: Pandas, NumPy, Scikit-learn
Tools: Git, GitHub, Jupyter Notebook, Google Colab, VS Code, PyCharm
Cloud: AWS (EC2, S3, IAM)
Core CS: Data Structures, Operating Systems, Computer Networks, OOP
Interpersonal Skills: Communication, Team Collaboration, Active Listening
Professional Skills: Problem Solving, Analytical Thinking, Adaptability, Time Management

PROJECTS
Real-Time Hand Tracking System
Python, OpenCV, Tkinter
Developed a Python-based real-time computer vision system using OpenCV with multithreaded
camera capture.
Optimized frame processing to improve stability and reduce latency by 25 percent.

IPL Cricket Analysis Dashboard
Power BI, DAX, Data Modeling
Analyzed 60000 plus IPL match records from 2008 to 2025 and built an interactive dashboard.
Designed KPIs, slicers, and advanced DAX measures to identify team and player performance trends.

Food Habits Awareness Community Service Project
Conducted a structured survey of 100 plus participants to analyze dietary habits and nutritional gaps.
Presented data-driven insights through awareness sessions to promote healthy food practices.

INTERNSHIP
Cloud Technology Intern AWS
Saraj Innotech Services Pvt Ltd
May 2025 to July 2025
Configured and managed AWS EC2, S3, and IAM services, reducing setup time by 20 percent.
Automated S3 backup and file management processes, cutting manual effort by 40 percent.
Optimized cloud resource usage using monitoring and networking tools.
Documented cloud configurations to support team collaboration and maintenance.

CERTIFICATIONS
Python Complete Course Udemy
Data Analysis and Decision Making NPTEL
NumPy for Data Science Udemy
Power BI Certification Infosys Springboard
Master Data Science and Machine Learning Masterclass GUVI and HCL
Machine Learning and Artificial Intelligence Beginners Course Udemy
Cloud Technology AWS Certification Saraj Innotech Services Pvt Ltd APSCHE and AICTE
"""  

In [ ]:
print(pred(myresume))